## Импорты

In [9]:
!pip install requests lxml pandas

import requests
import pandas as pd
import re
import gzip
import io

from lxml import html
from threading import Thread

## Датасет с описанием книг Children + Young/Adult

In [22]:
# Ссылки на оригинальные датасеты взяты с сайта: https://mengtingwan.github.io/data/goodreads
url_books_children = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/goodreads_books_children.json.gz"
url_books_adult = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/goodreads_books_young_adult.json.gz"

# Скачиваем 
json_books_children = str(gzip.decompress(requests.get(url_books_children).content), 'utf-8')
json_books_adult = str(gzip.decompress(requests.get(url_books_adult).content), 'utf-8')

In [24]:
# Объединяем
df_books_children = pd.read_json(io.StringIO(json_books_children), lines=True)
df_books_adult = pd.read_json(io.StringIO(json_books_adult), lines=True)
df_books = pd.concat([df_books_children, df_books_adult]).drop_duplicates(subset=['book_id']).reset_index(drop=True)
df_books

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,1599150603,7,[],US,,"[{'count': '56', 'name': 'to-read'}, {'count':...",,false,4.13,B00DU10PUG,...,9,,2006,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46,278578,The Aeneid for Boys and Girls,The Aeneid for Boys and Girls
1,1934876569,6,[151854],US,,"[{'count': '515', 'name': 'to-read'}, {'count'...",,false,4.22,,...,3,,2009,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,6066812,98,701117,All's Fairy in Love and War (Avalon: Web of Ma...,All's Fairy in Love and War (Avalon: Web of Ma...
2,0590417010,193,[],US,eng,"[{'count': '450', 'name': 'to-read'}, {'count'...",,false,4.43,B017RORXNI,...,9,,1995,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,1331,86259,Dog Heaven,Dog Heaven
3,0915190575,4,[],US,,"[{'count': '8', 'name': 'to-read'}, {'count': ...",,false,4.29,,...,,,,https://www.goodreads.com/book/show/3209312-mo...,https://s.gr-assets.com/assets/nophoto/book/11...,3209312,11,3242879,"Moths and Mothers, Feathers and Fathers: A Sto...","Moths and Mothers, Feathers and Fathers: A Sto..."
4,1416904999,4,[],US,,"[{'count': '8', 'name': 'to-read'}, {'count': ...",,false,3.57,,...,6,,2005,https://www.goodreads.com/book/show/1698376.Wh...,https://s.gr-assets.com/assets/nophoto/book/11...,1698376,23,1695373,What Do You Do?,What Do You Do?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214807,,3,[294897],US,tur,"[{'count': '2641', 'name': 'to-read'}, {'count...",,false,4.21,,...,,,2013,https://www.goodreads.com/book/show/18221503-l...,https://images.gr-assets.com/books/1387020971m...,18221503,21,17831603,"Ölmem Gerekirse (Revenants, #3)","Ölmem Gerekirse (Revenants, #3)"
214808,,9,[238215],US,en-US,"[{'count': '12325', 'name': 'to-read'}, {'coun...",B003YFIVYA,true,3.79,B003YFIVYA,...,11,,2010,https://www.goodreads.com/book/show/8987191-th...,https://images.gr-assets.com/books/1385945375m...,8987191,42,7100578,"The Mockingbirds (The Mockingbirds, #1)","The Mockingbirds (The Mockingbirds, #1)"
214809,,55,[],US,,"[{'count': '1684', 'name': 'to-read'}, {'count...",,false,3.78,,...,7,,2007,https://www.goodreads.com/book/show/1885730.Jo...,https://images.gr-assets.com/books/1376736258m...,1885730,1094,1886909,Joel and Cat Set the Story Straight,Joel and Cat Set the Story Straight
214810,0385756224,14,[540527],US,eng,"[{'count': '2401', 'name': 'to-read'}, {'count...",,true,3.67,,...,8,,2015,https://www.goodreads.com/book/show/23636536-a...,https://images.gr-assets.com/books/1418831526m...,23636536,29,26112061,Another Day,Another Day


In [25]:
# Анализируем некоторые столбцы
print(df_books.columns)

Index(['isbn', 'text_reviews_count', 'series', 'country_code', 'language_code',
       'popular_shelves', 'asin', 'is_ebook', 'average_rating', 'kindle_asin',
       'similar_books', 'description', 'format', 'link', 'authors',
       'publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'edition_information', 'publication_year', 'url',
       'image_url', 'book_id', 'ratings_count', 'work_id', 'title',
       'title_without_series'],
      dtype='object')


In [26]:
print(df_books['country_code'].value_counts().to_string())

country_code
US    214812


In [27]:
print(df_books['language_code'].value_counts().to_string())

language_code
         100919
eng       65533
en-US     11088
en-GB      4945
spa        3961
ger        3787
ind        2737
ita        2624
fre        2475
nl         2200
por        1927
fin        1377
swe        1132
cze        1027
tur         860
per         673
bul         651
dan         638
pol         536
gre         511
ara         495
en-CA       451
rum         422
vie         365
nor         365
slo         363
hun         341
rus         269
est         235
lav         200
scr         199
ben         152
msa         149
srp         143
lit         128
heb         127
cat          91
ukr          74
nob          71
tha          71
jpn          61
fil          49
kat          49
zho          44
mul          43
slv          26
pes          25
afr          16
en           14
nno          14
kor          14
isl          12
tam          11
mal          10
lat           8
tgl           8
sin           7
hye           7
bos           6
mkd           6
hin           5
sco       

In [28]:
print(df_books['format'].value_counts().to_string())

format
Paperback                                        73513
Hardcover                                        66218
                                                 38605
ebook                                            11835
Kindle Edition                                    8468
Audio CD                                          3257
Board Book                                        3214
Audiobook                                         2075
Mass Market Paperback                             1847
Unknown Binding                                    990
Audio                                              952
Library Binding                                    761
Audible Audio                                      743
Board book                                         565
Novelty Book                                       183
Broche                                             140
Audio Cassette                                     129
paperback                                           99
MP3

In [37]:
# Очищаем
df_books = df_books[['book_id', 'title', 'publication_year', 'language_code', 'format', 'description', 'url']]
df_books = df_books.replace('en-US', 'eng')
df_books = df_books.replace('en-GB', 'eng')
df_books = df_books.replace('en-CA', 'eng')
df_books = df_books[(df_books['language_code'] == 'eng') | (df_books['language_code'] == '')].reset_index(drop=True)
df_books

,book_id,title,publication_year,language_code,format,description,url
0,287141,The Aeneid for Boys and Girls,2006,,Paperback,"Relates in vigorous prose the tale of Aeneas, ...",https://www.goodreads.com/book/show/287141.The...
1,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,2009,,Paperback,"To Kara's astonishment, she discovers that a p...",https://www.goodreads.com/book/show/6066812-al...
2,89378,Dog Heaven,1995,eng,Hardcover,In Newbery Medalist Cynthia Rylant's classic b...,https://www.goodreads.com/book/show/89378.Dog_...
3,3209312,"Moths and Mothers, Feathers and Fathers: A Sto...",,,,,https://www.goodreads.com/book/show/3209312-mo...
4,1698376,What Do You Do?,2005,,Board Book,WHAT DO YOU DO?\nA hen lays eggs...\nA cow giv...,https://www.goodreads.com/book/show/1698376.Wh...
...,...,...,...,...,...,...,...
182931,16373114,"Abandon (Abandon, #1)",2011,eng,Audiobook,Though she tries returning to the life she kne...,https://www.goodreads.com/book/show/16373114-a...
182932,8987191,"The Mockingbirds (The Mockingbirds, #1)",2010,eng,,Some schools have honor codes.\nOthers have ha...,https://www.goodreads.com/book/show/8987191-th...
182933,1885730,Joel and Cat Set the Story Straight,2007,,Paperback,,https://www.goodreads.com/book/show/1885730.Jo...
182934,23636536,Another Day,2015,eng,ebook,The eagerly anticipated companion to David Lev...,https://www.goodreads.com/book/show/23636536-a...


In [38]:
print(df_books['language_code'].value_counts().to_string())

language_code
       100919
eng     82017


In [40]:
# Сохраняем
df_books.to_json("data/books.json", orient="records", lines=True)

## Датасет с вопросами пользователей из треда Children + Young/Adult
### https://www.goodreads.com/topic/group_folder/988

In [10]:
# Подготовка к парсингу сайта через XPATH
# XPATH - путь до объекта в HTML структуре

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36', 'Accept-Encoding': 'gzip'}

main_page_url_base = 'https://www.goodreads.com'
main_page_url_folder = 'https://www.goodreads.com/topic/group_folder/988?page='
main_page_xpath = '//tr/td/a[@class=""]/@href'

topic_page_xpath_query = '/html/body/div[2]/div[3]/div[1]/div[1]/div[2]/div[5]/div[2]/div[3]/text()'
topic_page_xpath_answer = '/html/body/div[2]/div[3]/div[1]/div[1]/div[2]/div[1]/a/@href'

In [ ]:
# Пробегаемся по решенным топикам, вытаскивая запросы пользователей и официальный (промодерированный) ответ

df_books = pd.read_json('data/books.json', lines=True)
df_queries = pd.read_json('data/queries.json', lines=True)
pages_count = 4 # 78 per 30 book
threads_count = 2 # 6

# Основной скрипт
def parse(pages_indexes):
    global df_queries
    global df_books

    for i in pages_indexes:
        main_page_url = main_page_url_folder + str(i)
        print("parsing: " + main_page_url)

        main_page_response = requests.get(main_page_url, headers=headers)
        main_page_tree = html.fromstring(main_page_response.content)
        main_page_topic_urls = map(lambda x: main_page_url_base + x, main_page_tree.xpath(main_page_xpath))

        for topic_page_url in main_page_topic_urls:

            # get topic page
            topic_page_response = requests.get(topic_page_url, headers=headers)
            topic_page_tree = html.fromstring(topic_page_response.content)

            # parse query
            topic_query = topic_page_tree.xpath(topic_page_xpath_query)
            topic_query = ''.join(topic_query).strip().replace('â\x80\x99', "'")
            re.sub(r"\s+", " ", topic_query)
            if topic_query == None or topic_query == "" or len(topic_query) < 10: continue

            # parse book url
            book_url = topic_page_tree.xpath(topic_page_xpath_answer)
            book_url = book_url[0].split('/')[-1]
            if book_url == None or book_url == "": continue
            book_url = main_page_url_base + book_url

            # find book id from book dataset
            book_id = df_books[df_books['url'] == book_url]['book_id'].values()
            if (len(book_id) == 0): continue
            book_id = book_id[0]
            if (book_id in list(df_queries['book_id'])): continue
            df_queries.loc[len(df_queries)] = [book_id, topic_query]

            print(f"ADDED: {book_id}")


# Многопоточный запуск для ускорения парсинга
# threads = []
# for i in range(threads_count):
#     step = pages_count // threads_count
#     threads.append(Thread(target=main, 
#                           args=[
#                              list(range((i * step) + 1, ((i + 1) * step) + 1))
#                              ]
#                           ))
#     threads[-1].start()
# for t in threads:
#     t.join()


parse([1])
df_queries.to_json("data/queries.json", lines=True)
df_queries